In [9]:
import importlib
import sys

# Remove any cached modules
if 'src.ucb_kd_trainer' in sys.modules:
    del sys.modules['src.ucb_kd_trainer']

import datasets
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from datasets import load_dataset
from src.ABMIL_UCB import ABMIL_UCB
from tqdm import tqdm
import numpy as np
import os
from src.ucb_kd_trainer import ABMILUCBTrainingArguments, ABMILFeatureDataset, create_abmil_ucb_trainer


In [10]:
dataset = datasets.load_from_disk('/equilibrium/datasets/TCGA-histological-data/huggingfac/KN_dataset')

In [11]:
train_dataset = ABMILFeatureDataset(dataset['train']['features'], dataset['train']['targets'], num_features=512)
test_dataset = ABMILFeatureDataset(dataset['test']['features'], dataset['test']['targets'], num_features=512)

In [12]:
model = ABMIL_UCB(
        feature_dim=768,
        head_dim=256,
        n_heads=8,
        dropout=0.2,
        n_branches=1,
        gated=False
    )

In [13]:
training_args = ABMILUCBTrainingArguments(
        output_dir='./results',
        num_train_epochs=10,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=100,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=10,
        eval_strategy="steps",
        eval_steps=50,
        save_strategy="steps",
        save_steps=100,
        load_best_model_at_end=True,
        metric_for_best_model="accuracy",
        greater_is_better=True,
        # ABMIL UCB specific parameters
        num_features=196,
        ucb_enabled=True,
        ucb_beta=1.0,
        ucb_warmup_iter=500,
        ucb_top_k=10,
        n_heads=8,
        n_branches=1,
        remove_unused_columns=False
    )

In [14]:
trainer = create_abmil_ucb_trainer(
        model=model,
        training_args=training_args,
        train_dataset=train_dataset,
        eval_dataset=test_dataset
    )
    

In [15]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy,Mean Similarity,Std Similarity
50,0.173200,0.134890,0.865110,0.865110,0.079289
100,0.078800,0.084468,0.915532,0.915532,0.047065


/home/vcivale/WSI-RL-Tiles-Selection_3/.venv/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at /pytorch/aten/src/ATen/cuda/CublasHandlePool.cpp:181.)
  return F.linear(input, self.weight, self.bias)


TrainOutput(global_step=120, training_loss=0.27743588040272393, metrics={'train_runtime': 57.7546, 'train_samples_per_second': 16.276, 'train_steps_per_second': 2.078, 'total_flos': 0.0, 'train_loss': 0.27743588040272393, 'epoch': 10.0})